In [1]:
import pandas
import numpy
import pygsheets

In [2]:
pandas.__version__

'1.5.0'

# credentials info

In [3]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1v38EY_kFFsT17cPqz39j21zRYWeQSSxuDmz565MD2GU') # copy for 2022 report

# sheet 1 (old)

In [20]:
#spreadsheet1 = gc.open_by_key('1MrghwBeCz8Tzgua7CWGg_KoXKVZsV7r0kHMYHYqnNTg') # Aug 18 version
spreadsheet1 = gc.open_by_key('129b9YeQn7uAIsMgyB5uleT2QZXYxjD_UetnqwVyoRaI') # Aug 19 version, official database

#spreadsheet[1] "Gas Pipelines" tab is the second index
terms_df_orig1 = spreadsheet1.worksheet('title', 'Terminals').get_as_df(start='A2')

In [21]:
# replace all -- with nans
terms_df_orig1.replace('--', numpy.nan, inplace=True)
# remove oil export terminals
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Fuel']=='LNG']
# remove anything without a wiki page
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Wiki']!='']
# remove N/A statuses
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Status']!='']

# sheet 2 (new)

In [23]:
spreadsheet2 = gc.open_by_key('1fNf3eK_nqgzXX6ijGlg9A4MPXAF4fh9ZmH3XnsrvS9A') # copy for mid-Sep 2022 terminals briefing
terms_df_orig2 = spreadsheet2.worksheet('title', 'Terminals').get_as_df(start='A2')

In [24]:
# replace all -- with nans
terms_df_orig2.replace('--', numpy.nan, inplace=True)
# remove oil export terminals
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Fuel']=='LNG']
# remove anything without a wiki page
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Wiki']!='']
# remove N/A statuses
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Status']!='']

# compare

In [25]:
terms_df_orig2.columns==terms_df_orig1.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True])

In [26]:
set(terms_df_orig1.columns.tolist())-set(terms_df_orig2.columns.tolist())

{'Rob'}

## get common columns

In [47]:
common_columns = list(set(terms_df_orig1.columns.tolist()).intersection(set(terms_df_orig2.columns.tolist())))

In [48]:
terms_df_orig1.shape

(1173, 76)

In [49]:
terms_df_orig2.shape

(1175, 76)

## get different ComboIDs

In [29]:
diff_comboids = list(set(terms_df_orig2.ComboID)-set(terms_df_orig1.ComboID))

In [30]:
terms_df_orig1.loc[terms_df_orig1.ComboID.isin(diff_comboids)]

,TerminalID,ProjectID,ComboID,Wiki,TerminalName,UnitName,Status,Fuel,Import/Export,Country,...,ResearcherNotes3,ExportFigureClarkWilliamsDerry,IGUWorldLNGReportAppendix2,Rob,CostUSDPerBcm/y,CostUSDPerMtpa,ProposalYearMonth,FIDYearMonth,ConstructionYearMonth,StartYear1YearMonth


In [32]:
terms_df_orig2.loc[terms_df_orig2.ComboID.isin(diff_comboids)]

,TerminalID,ProjectID,ComboID,Wiki,TerminalName,UnitName,Status,Fuel,Import/Export,Country,...,ResearcherNotes3,ExportFigureClarkWilliamsDerry,IGUWorldLNGReportAppendix2,Construction_Sanctioned_July_2022,CostUSDPerBcm/y,CostUSDPerMtpa,ProposalYearMonth,FIDYearMonth,ConstructionYearMonth,StartYear1YearMonth
1216,T1088,3,T108803,https://www.gem.wiki/Fos_Cavaou_LNG_Terminal,Fos Cavaou LNG Terminal,2022 Expansion,Proposed,LNG,Import,France,...,,,,,NaN,NaN,,,,
1217,T1089,4,T108904,https://www.gem.wiki/Fos_Cavaou_LNG_Terminal,Fos Cavaou LNG Terminal,2023 Expansion,Proposed,LNG,Import,France,...,,,,,NaN,NaN,,,,


## if there are diff ComboIDs, pull out only the common ones, THEN compare rest

In [38]:
common_comboids = list(set(terms_df_orig2.ComboID).intersection(set(terms_df_orig1.ComboID)))

In [50]:
common_df1 = terms_df_orig1.loc[terms_df_orig1.ComboID.isin(common_comboids),common_columns]
common_df2 = terms_df_orig2.loc[terms_df_orig2.ComboID.isin(common_comboids),common_columns]

In [57]:
index_list = common_df2.compare(common_df1, result_names=('2','1')).index
common_df2.compare(common_df1, result_names=('2','1'))

Capacity       LastUpdated             StartYearEarliest        FID       \
            2     1           2           1                 2     1    2    1   
152     13.33  10.0         NaN         NaN               NaN   NaN  NaN  NaN   
216       NaN   NaN         NaN         NaN               NaN   NaN  FID        
225       NaN   NaN  2022/08/23  2022/06/06              2029  2027  NaN  NaN   
226       NaN   NaN  2022/08/23  2022/06/06               NaN   NaN  NaN  NaN   
372       NaN   NaN  2022/09/05  2022/05/20              2029  2027  NaN  NaN   
514       NaN   NaN  2022/09/06  2022/06/23              2024  2029  NaN  NaN   
873       NaN   NaN  2022/09/06  2022/06/24               NaN   NaN  NaN  NaN   
1116      NaN   NaN  2022/09/13  2022/07/21               NaN   NaN  NaN  NaN   

     CapacityInMtpa        ... FIDYear            CapacityInBcm/y        \
                  2     1  ...       2          1               2     1   
152           13.33  10.0  ...     NaN        NaN           18.13  13.6   
216             NaN   NaN  ...    2020                        NaN   NaN   
225             NaN   NaN  ...    2025  2021/2022             NaN   NaN   
226             NaN   NaN  ...    2025  2021/2022             NaN   NaN   
372             NaN   NaN  ...     NaN        NaN             NaN   NaN   
514             NaN   NaN  ...     NaN        NaN             NaN   NaN   
873             NaN   NaN  ...     NaN        NaN             NaN   NaN   
1116            NaN   NaN  ...     NaN        NaN             NaN   NaN   

     StartYear1       CostUSDPerBcm/y              StartYear2       
              2     1               2            1          2    1  
152         NaN   NaN     728123275.3  970588325.9        NaN  NaN  
216         NaN   NaN             NaN          NaN        NaN  NaN  
225        2029  2027             NaN          NaN       2030       
226         NaN   NaN             NaN          NaN        NaN  NaN  
372        2029  2027             NaN          NaN        NaN  NaN  
514        2024  2029             NaN          NaN       2025       
873         NaN   NaN             NaN          NaN        NaN  NaN  
1116        NaN   NaN             NaN          NaN        NaN  NaN  

[8 rows x 26 columns]

In [75]:
diff_df = common_df2.compare(common_df1, result_names=('new','old'))

In [76]:
diff_df['ComboID'] = numpy.nan

In [77]:
diff_df.loc[:,'ComboID'] = common_df1.loc[diff_df.index,'ComboID']
diff_df.loc[:,'TerminalName'] = common_df1.loc[diff_df.index,'TerminalName']

In [78]:
diff_df.to_excel('differences.xlsx')#, index=False)

In [79]:
common_df1.loc[diff_df.index,['ComboID','TerminalName']]

,ComboID,TerminalName
152,T024201,Plaquemines LNG Terminal
216,T025913,Nigeria LNG Terminal
225,T026301,Tanzania LNG Terminal
226,T026302,Tanzania LNG Terminal
372,T035001,Cedar FLNG Terminal
514,T043001,Krk FSRU
873,T070701,Portovaya LNG Terminal
1116,T099700,Eemshaven FSRU
